In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import keras

Using TensorFlow backend.


In [2]:
#read data
df = pd.read_csv("Three_Emoji_Simplified_root.csv")
test_df = pd.read_csv('test_set_root.csv')

train_df = df.copy().iloc[:,2:].astype(float)
test_df = test_df.copy().iloc[:,1:].astype(float)

In [3]:
x_train = train_df.iloc[:,:-3].values
y_train = train_df.iloc[:,-3:].values

x_test = test_df.iloc[:,:-3].values
y_test = test_df.iloc[:,-3:].values


y_train_label = np.apply_along_axis(lambda x: np.where(x == 1), 1, y_train).squeeze()
y_test_label = np.apply_along_axis(lambda x: np.where(x == 1), 1, y_test).squeeze()

In [5]:
from collections import Counter
Counter(test_df[:-3].sum(1))

Counter({1.0: 85,
         2.0: 227,
         3.0: 364,
         4.0: 371,
         5.0: 363,
         6.0: 349,
         7.0: 319,
         8.0: 309,
         9.0: 301,
         10.0: 247,
         11.0: 259,
         12.0: 224,
         13.0: 186,
         14.0: 107,
         15.0: 74,
         16.0: 51,
         17.0: 20,
         18.0: 2,
         19.0: 9,
         20.0: 5,
         21.0: 4,
         22.0: 1,
         23.0: 1,
         24.0: 1,
         35.0: 1})

In [6]:
test_mask_2 = test_df[:-3].sum(1) > 7
fil_df = test_df.copy().iloc[test_mask_2.values,:]
x_test_fil = fil_df.iloc[:,:-3].values
y_test_fil = fil_df.iloc[:,-3:].values

y_test_fil_label = np.apply_along_axis(lambda x: np.where(x == 1), 1, y_test_fil).squeeze()

# Model Structure

In [7]:
def add_layer(inputs, in_size, out_size, act_fun = None):
    W = tf.Variable(tf.random_normal([in_size, out_size]),
                   )
    b = tf.Variable(tf.zeros([1, out_size]) + 0.1,
                   )
    z = tf.matmul(inputs, W) + b
    
    if act_fun == None:
        outputs = z
    else:
        outputs = act_fun(z)
    
    return outputs

In [8]:
def next_batch(num, data, labels):

    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = data[idx]
    labels_shuffle = labels[idx]
    #labels_shuffle = np.asarray(labels_shuffle.values.reshape(len(labels_shuffle), 1))

    return data_shuffle, labels_shuffle

In [52]:
import tensorflow as tf

xs = tf.placeholder(tf.float32, [None, x_train.shape[1]])
ys = tf.placeholder(tf.float32, [None, y_train.shape[1]])

keep_prob = tf.placeholder(tf.float32)

In [74]:
#hidden layer
l1 = tf.nn.dropout(add_layer(xs, x_train.shape[1], 10, act_fun = tf.nn.relu), keep_prob)
#l1 = add_layer(xs, x_train.shape[1], 10, act_fun = tf.nn.relu)

#hidden layer 2
l2 = tf.nn.dropout(add_layer(l1, 10, 10, act_fun = tf.nn.relu), keep_prob)
#l2 = add_layer(l1, 10, 10, act_fun = tf.nn.relu)

#hidden layer 3
l3 = tf.nn.dropout(add_layer(l2, 10, 10, act_fun = tf.nn.relu), keep_prob)

l4 = tf.nn.dropout(add_layer(l3, 10, 10, act_fun = tf.nn.relu), keep_prob)

l5 = add_layer(l4, 10, 10, act_fun = tf.nn.relu)

#output layer
y_hat = add_layer(l2, 10, 3, act_fun = tf.nn.softmax)

In [75]:
# Define loss fun.
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels = ys,
                                           logits = y_hat))

# Define a optimizer
train_step_adam = tf.train.AdamOptimizer(
    learning_rate = 0.01).minimize(loss)

# Convert logits to label indexed
correct_pred = tf.argmax(y_hat, 1)

# Define an accuracy matric
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [ ]:
from __future__ import division

tf.set_random_seed(1234)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

num_of_step = 20001

for i in range(num_of_step):
    x_batch, y_batch = next_batch(512, x_train, y_train)
    
    accuracy_val = sess.run([accuracy], feed_dict={xs:x_batch, ys:y_batch, keep_prob:1.0})
    _ = sess.run([train_step_adam], feed_dict={xs:x_batch, ys:y_batch, keep_prob:0.5})
    #_, accuracy_val = sess.run([train_step_adam, accuracy], 
    #            feed_dict={xs:x_batch, ys:y_batch, keep_prob:0.5})
    
    if i % 1000 == 0:
        print('At iteration: ' + str(i + 1))
        print('Loss: ' + str(sess.run(loss, feed_dict={xs:x_train, ys:y_train,keep_prob:1.0})))
        print('')
    
    
    if i % 2000 == 0:
        train_predicted = sess.run([correct_pred], 
                     feed_dict={xs: x_train,keep_prob:1.0})[0]
        test_predicted = sess.run([correct_pred], 
                     feed_dict={xs: x_test_fil,keep_prob:1.0})[0]
        train_outcome = (pd.get_dummies(train_predicted) == y_train)
        test_outcome = (pd.get_dummies(test_predicted) == y_test_fil)
        
        train_accuracy = np.sum(train_outcome.apply(sum, axis=1) == 3) / len(y_train)
        test_accuracy = np.sum(test_outcome.apply(sum, axis=1) == 3) / len(y_test_fil)

        
        print('Train Accuracy: ' + str(train_accuracy * 100) + ' %')
        print('Test Accuracy: ' + str(test_accuracy * 100) + ' %') 
        print('')
        
    if (train_accuracy - test_accuracy) > 0.5:
        #print('At epoch: ' + str(i + 1))
        print('Diff. between over 30% -> Overfitting')
        break
        
    if test_accuracy > 0.7:
        #print('At epoch: ' + str(i + 1))
        print('Test Accuracy reached ' + str(test_accuracy * 100) + ' %')
        break
    
    #tf.confusion_matrix(y_test_fil, test_predicted)
#sess.close()


At iteration: 1
Loss: 1.2076268

Train Accuracy: 33.09568931805192 %
Test Accuracy: 34.01775804661487 %

At iteration: 1001
Loss: 1.0979724

At iteration: 2001
Loss: 1.0836782

Train Accuracy: 37.86361038414102 %
Test Accuracy: 36.90344062153163 %

At iteration: 3001
Loss: 1.0551473

At iteration: 4001
Loss: 1.0316415

Train Accuracy: 46.039767770658486 %
Test Accuracy: 30.077691453940066 %

At iteration: 5001
Loss: 1.0232543

At iteration: 6001
Loss: 1.0159816

Train Accuracy: 46.5661943867882 %
Test Accuracy: 36.015538290788015 %



In [15]:
fitted = sess.run([correct_pred], 
                     feed_dict={xs: x_train})[0]
predicted = sess.run([correct_pred], 
                     feed_dict={xs: x_test_fil})[0]

#print(predicted)

from __future__ import division
outcome_train = pd.get_dummies(fitted) == y_train
outcome_test = pd.get_dummies(predicted) == y_test_fil

train_accuracy = np.sum(outcome_train.apply(sum, axis=1) == 3) / len(y_train)
test_accuracy = np.sum(outcome_test.apply(sum, axis=1) == 3) / len(y_test_fil)

print('Test Accuracy: ' + str(train_accuracy * 100) + ' %')
print('Test Accuracy: ' + str(test_accuracy * 100) + ' %')

#sess.close()

[1 1 2 ... 2 0 0]
Test Accuracy: 32.46392896781354 %


In [ ]:
'''
Outcome: 
lr=0.01
201 2 layer[in, 3, out]: 62.3%
501 2 layer[in, 10, out]: 64.9%
101 2 layer[in, 100, out]: 54.5%
501 3 layer[in, 10, 10, out]: 68.39%
501 3 layer[in, 20, 20, out]: 60.39%
501 3 layer[in, 50, 50, out]: 55.70%
101 3 layer[in, 100, 100, out]: 40.8% 
10001 3 layer[6603, 10, 10, 10, out]: 58.9 /33.22
20001 3 layer[6603, 15, 15, out]: 62% /36%
10001 3 layer[5587, 10, 10, 3] Adagrad: 

10001 3 layer[6603, 10, 10, 3], lr=0.05: 40%/30%

batch size = 512
10001 [5587, 10, 10, 3], 0.9 dropout: 61/32
10001 [5587, 10, 10, 3], 0.5 dropout: 46/36 both increased smoothly
10001 [5587, 10, 10, 10, 3], 0.5 dropout: 48/34 inc/ random 

batch size = 256
10001 [5587, 10, 10, 3], 0.5 dropout:45, 32 inc/random

當units數為50 ~ 100(lr 0.05)時，從500開始會被困在local optima
'''

# Keras

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense
from sklearn.model_selection import train_test_split
import numpy as np

seed = 2
np.random.seed(seed)

dataset = np.loadtxt('Three_Lable_Simplified_root.csv', delimiter=',',skiprows=1)

In [ ]:
model = Sequential()
model.add(Dense(5, input_dim=, init='uniform', activation='relu'))
model.add(Dense(5, init='uniform', activation='relu'))
model.add(Dense(3, init='uniform', activation='sigmoid'))

model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [ ]:
model.fit(X_train, Y_train,
          validation_data=(X_test, Y_test)
          nb_epoch=100, batch_size=100)